In [1]:
"""
Clasificador few-shot usando Gemini + ingeniería de prompts.

- El modelo devolverá etiquetas y confianza en formato JSON.
"""

# Si ya tienes instalada la librería en la celda anterior, puedes omitir esta línea
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata
import os
import json
import pandas as pd
import time

# -------------------------------------------------------------------
# 0. Configurar API de Gemini
# -------------------------------------------------------------------
API_KEY = userdata.get('GOOGLE_API_KEY')

if API_KEY is None:
    raise ValueError(
        "No se encontró la API Key. Por favor, guárdala en los secretos de Colab "
        "bajo el nombre 'GOOGLE_API_KEY' o 'GEMINI_API_KEY'."
    )

genai.configure(api_key=API_KEY)
print("API de Gemini configurada exitosamente.")

# Elegir modelo
GEMINI_MODEL_NAME = "gemini-2.5-flash"
model = genai.GenerativeModel(GEMINI_MODEL_NAME)

API de Gemini configurada exitosamente.


In [2]:


# -------------------------------------------------------------------
# 1. Cargar CSV
# -------------------------------------------------------------------
# Asegúrate de que 'test1.csv' esté subido al entorno de Colab
df = pd.read_csv("test1.csv", encoding="utf-8", sep=",")
print("DataFrame cargado. Filas:", len(df))
print(df.columns)

# -------------------------------------------------------------------
# 2. Definiciones de categorías
# -------------------------------------------------------------------
DEFINICIONES_DE_CATEGORIAS = {
    "servicio de alcantarillado u otras formas de disposicion sanitaria de excretas": """
Texto describe al menos uno (1) de los siguientes servicios:

1. SERVICIO DE ALCANTARILLADO: incluye al menos uno (1) de los siguientes procesos con aguas residuales:
   recolección, impulsión, conducción hasta el punto de entrega y tratamiento.

2. SERVICIO DE DISPOSICIÓN SANITARIA DE EXCRETAS: incluye instalaciones intradomiciliarias
   (como Unidades Básicas Sanitarias - UBS, letrinas, etc.) y/o procesos para la disposición final del agua residual
   o las excretas.
""",
  "servicio de agua potable mediante red publica o pileta publica": """
comprende al menos uno (1) de los siguientes servicios:
1. sistema de producción: captación, almacenamiento y conducción de agua cruda, tratamiento y conducción de agua potable.
2. sistema de distribución: almacenamiento, distribución, entrega y medición al usuario.
comprende instalaciones como:
1. red pública: conexión domiciliaria.
2. pileta públicas: punto de abastecimiento de agua potable instalado en el espacio público.
"""
}

DataFrame cargado. Filas: 5
Index(['NOMBRE DEL PROYECTO_limpio', 'BRECHA_limpio', 'NATURALEZA_PROYECTO',
       'UBICACION_GEOGRAFICA', 'BRECHA_limpio2', 'URBANO_RURAL', 'ID'],
      dtype='object')


In [3]:
# -------------------------------------------------------------------
# 3. Construcción del prompt
# -------------------------------------------------------------------
def construir_prompt(row: pd.Series) -> str:
    """
    Construye un prompt en español para que Gemini actúe como
    clasificador multi-etiqueta basado en las definiciones de categorías.
    """
    texto_proyecto = str(row.get("NOMBRE DEL PROYECTO_limpio", "")).strip()

    partes_categorias = []
    for nombre, definicion in DEFINICIONES_DE_CATEGORIAS.items():
        partes_categorias.append(f"- {nombre}: {definicion.strip()}")
    texto_categorias = "\n".join(partes_categorias)

    prompt = f"""
Eres un modelo de lenguaje experto en clasificación de títulos de proyectos públicos
según brechas de infraestructura y servicios definidas por el SNPMGI del Perú.

Tu única tarea es leer el título del proyecto y asignarle UNA O VARIAS categorías
de brecha de la lista definida abajo.

CATEGORÍAS DISPONIBLES:
{texto_categorias}

REGLAS ESTRICTAS:
- Analiza el significado del título del proyecto, no solo palabras sueltas.
- Asigna múltiples categorías solo si el título realmente cubre más de una brecha.
- No inventes información adicional que no esté presente o inferida razonablemente del título del proyecto.
- Si el título es ambiguo o no coincide con ninguna definición, devuelve "labels": [].

- Genera un valor "confianza" entre 0.0 y 1.0.
- Genera una "justificacion" de máximo 200 palabras basada únicamente en las definiciones dadas.

FORMATO DE RESPUESTA (OBLIGATORIO):
Responde ÚNICAMENTE con JSON válido, sin texto adicional, sin explicaciones, sin backticks.
Ejemplo de formato:

{{
  "labels": [
    {{
      "label": "NOMBRE_DE_CATEGORIA_1",
      "confianza": 0.95,
      "justificacion": "Texto de la justificación"
    }}
  ]
}}

Si no hay categorías aplicables:

{{
  "labels": []
}}

TEXTO A CLASIFICAR:
\"\"\"{texto_proyecto}\"\"\"
"""
    return prompt

# -------------------------------------------------------------------
# 4. Utilidad para limpiar y extraer JSON de la respuesta del modelo
# -------------------------------------------------------------------
def extraer_json_de_texto(texto: str) -> str:
    """
    Intenta limpiar la salida del modelo y extraer solo la parte JSON.

    - Elimina backticks de bloques de código (```json ... ```).
    - Toma el contenido entre la primera '{' y la última '}'.
    - Valida que sea JSON.
    """
    if texto is None:
        raise json.JSONDecodeError("Respuesta vacía del modelo", "", 0)

    s = texto.strip()

    # 1) Quitar bloques de código markdown si los hubiera
    if s.startswith("```"):
        # eliminar primera línea (``` o ```json)
        first_newline = s.find("\n")
        if first_newline != -1:
            s = s[first_newline + 1:]
        # eliminar ``` final si está
        if s.endswith("```"):
            s = s[:-3]
        s = s.strip()

    # 2) Quedarnos con lo que está entre la primera '{' y la última '}'
    start = s.find("{")
    end = s.rfind("}")
    if start != -1 and end != -1 and end > start:
        s = s[start:end + 1]

    # 3) Intentar parsear como JSON
    obj = json.loads(s)  # si falla, lanza JSONDecodeError
    # 4) Devolverlo normalizado (string JSON)
    return json.dumps(obj, ensure_ascii=False)

# -------------------------------------------------------------------
# 5. Función para llamar a Gemini y devolver JSON limpio
# -------------------------------------------------------------------
def ejecutar_prompt(prompt: str, max_reintentos: int = 3, espera_segundos: int = 2) -> str:
    """
    Envía el prompt al modelo de Gemini y devuelve la respuesta como JSON limpio (string).
    Si la respuesta no es JSON válido, devuelve un JSON de error.
    """
    for intento in range(1, max_reintentos + 1):
        try:
            respuesta = model.generate_content(prompt)
            texto = (respuesta.text or "").strip()

            try:
                json_limpio = extraer_json_de_texto(texto)
                return json_limpio
            except json.JSONDecodeError as e:
                # Si el modelo no respetó el formato, devolvemos error pero guardamos la respuesta cruda
                return json.dumps({
                    "labels": [],
                    "error": "La respuesta del modelo no es JSON válido",
                    "detalle_error": str(e),
                    "raw_response": texto
                }, ensure_ascii=False)

        except Exception as e:
            print(f"[Intento {intento}] Error al llamar a Gemini: {e}")
            time.sleep(espera_segundos)

    # Si falla todos los intentos, devolvemos un JSON de error
    return json.dumps({
        "labels": [],
        "error": "No se pudo obtener respuesta del modelo luego de varios intentos."
    }, ensure_ascii=False)


In [4]:
# -------------------------------------------------------------------
# 6. Aplicar el modelo fila por fila y guardar en RESPUESTA_PROMPT
# -------------------------------------------------------------------
def clasificar_fila(row: pd.Series) -> str:
    prompt = construir_prompt(row)
    respuesta = ejecutar_prompt(prompt)
    return respuesta

df["RESPUESTA_PROMPT"] = df.apply(clasificar_fila, axis=1)

# -------------------------------------------------------------------
# 7. Imprimir el DataFrame resultante
# -------------------------------------------------------------------
# print(df[["NOMBRE DEL PROYECTO_limpio", "RESPUESTA_PROMPT"]])

In [5]:
# Guardar el DataFrame en un archivo CSV
df.to_csv("mi_dataframe.csv", index=False)

print("Archivo CSV creado correctamente.")


Archivo CSV creado correctamente.
